# Retrieval-Augmented Generation (RAG)

In [2]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [3]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [4]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

/Users/ericmurray/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 34.0MiB/s]


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [13]:
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

In [15]:
results = nutrition_qna.query(query_texts=["precnancy"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('is_pregnancy', True)]
Question: What role does early preganncy detection play in maternal care?
        Answer: Early preganncy detection plays a crucial role in maternal care by facilitating early identification of any health issues, enabling prompt treatment, and improving overall pregnancy outcomes.

        This Q&A pair provides information about nutrition and health topics.


[('is_pregnancy', True)]
Question: How does early recognition of preganncy contribute to the well-being of the baby?
        Answer: Early recognition of pregnancy contributes to the well-being of the baby by allowing for early detection of any potential issues, ensuring that appropriate care is provided and improving neonatal outcomes.

        This Q&A pair provides information about nutrition and health topics.




In [29]:
results = nutrition_qna.query(query_texts=["pregnancy"], n_results=2)
results


{'ids': [['qa_2664', 'qa_2950']],
 'embeddings': None,
 'documents': [['Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?\n        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.\n\n        This Q&A pair provides information about nutrition and health topics.',
   'Question: What health issues can make a pregnancy more challenging?\n        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.\n\n        This Q&A pair provides information about nutrition and health topics.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': 

In [40]:
results['documents'][0]

['Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?\n        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.\n\n        This Q&A pair provides information about nutrition and health topics.',
 'Question: What health issues can make a pregnancy more challenging?\n        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.\n\n        This Q&A pair provides information about nutrition and health topics.']

In [ ]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition Q and A found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [44]:
@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up nutrition questions and answers
    about commen health concerns and nutrition topics.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition questiona and their answers.
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        qna = doc

        formatted_results.append(qna)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [ ]:
# print(calorie_lookup_tool('fish'))


Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [45]:
print(nutrition_qna_tool('pregnancy'))

TypeError: 'FunctionTool' object is not callable

In [46]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If you need to look up questions related to health and nutrion, use the nutrition_qna_tool.
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool],
)

In [47]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many eggs are in a bakers dozen?",
    )
    print(result.final_output)

13 eggs.


In [48]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

- Total calories (one banana + one apple): ~200 kcal
- Per 100 g:
  - Banana: 89 kcal
  - Apple: 52 kcal

Notes: A medium banana ~118 g (~105 kcal); a medium apple ~182 g (~95 kcal). Totals are approximate.


In [49]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many extra calories should I eat if I am pregnant",
    )
    print(result.final_output)

General guidance:
- Second half of pregnancy: about 300 extra calories per day.
- Early pregnancy (first trimester): often no extra calories needed, or only a modest amount (around 150–350 extra if advised).
- Total guidance varies by individual, so check with your healthcare provider.

If you’d like, I can tailor this to your trimester and weight gain plan.
